## Data Ingestion

In [26]:
import pandas as pd

# column_names = ['id', 'text', 'label']
twitter_train = pd.read_csv("/kaggle/input/dataset-3/train_formatted.csv")
twitter_test = pd.read_csv("/kaggle/input/dataset-3/test_formatted.csv")
twitter_dev = pd.read_csv("/kaggle/input/dataset-3/dev_formatted.csv")

In [27]:
twitter_train.shape, twitter_test.shape, twitter_dev.shape

((1557, 5), (520, 5), (485, 5))

## Data Preparation

In [28]:
LABELS = twitter_train['label'].unique().tolist()
train_label = twitter_train['label'].replace(LABELS, [0, 1, 2, 3]).tolist()

/tmp/ipykernel_36/3616623573.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_label = twitter_train['label'].replace(LABELS, [0, 1, 2, 3]).tolist()


In [29]:
train_data = twitter_train['text'].tolist()

In [30]:
dev_data = twitter_dev['text'].tolist()
dev_label = twitter_dev['label'].replace(LABELS, [0, 1, 2, 3]).tolist()

/tmp/ipykernel_36/3987848631.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dev_label = twitter_dev['label'].replace(LABELS, [0, 1, 2, 3]).tolist()


In [31]:
test_data = twitter_test['text'].tolist()
test_label = twitter_test['label'].replace(LABELS, [0, 1, 2, 3]).tolist()

/tmp/ipykernel_36/2548705681.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_label = twitter_test['label'].replace(LABELS, [0, 1, 2, 3]).tolist()


In [32]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class ClassificationDataset(Dataset):
    def __init__(self, texts: list[str], labels: list[int], tokenizer, max_length: int):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text, 
            add_special_tokens=True, 
            max_length=self.max_length, 
            return_token_type_ids=False, 
            padding="max_length",
            truncation=True, 
            return_attention_mask=True, 
            return_tensors="pt"
        )
        
        return {
            "input_ids": encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [33]:
from transformers import BertTokenizer, BertModel, AdamW

max_length = 128

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_dataset = ClassificationDataset(train_data, train_label, tokenizer, max_length)
dev_dataset = ClassificationDataset(dev_data, dev_label, tokenizer, max_length)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [34]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)

In [35]:
test_dataset = ClassificationDataset(test_data, test_label, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## Model Preparation

In [36]:
num_classes = 4

In [37]:
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, num_classes, dropout=0.1):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(dropout)
        self.ffn = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.ffn(x)
        return logits

In [38]:
bert_model = BertModel.from_pretrained('bert-base-uncased')

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [39]:
model = BERTClassifier(bert_model, num_classes)

In [40]:
learning_rate = 2e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [41]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
from tqdm import tqdm

def calculate_metrics(y_true, y_pred, y_pred_proba, num_classes):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted'),
        'recall': recall_score(y_true, y_pred, average='weighted'),
        'f1': f1_score(y_true, y_pred, average='weighted')
    }
    
    # Calculate ROC-AUC score
    if num_classes == 2:
        metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba[:, 1])
    else:
        try:
            metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average='weighted')
        except:
            metrics['roc_auc'] = None
            
    return metrics

In [42]:
def evaluate(model, data_loader, device, num_classes):
    model.eval()
    all_labels = []
    all_predictions = []
    all_predictions_proba = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            all_predictions_proba.extend(probabilities.cpu().numpy())

    all_labels = np.array(all_labels)
    all_predictions = np.array(all_predictions)
    all_predictions_proba = np.array(all_predictions_proba)

    return calculate_metrics(all_labels, all_predictions, all_predictions_proba, num_classes)

In [43]:
def train(model, train_loader, val_loader, criterion, optimizer, device, num_epochs):
    best_val_metrics = {'f1': 0.0}
    history = {'train': [], 'val': []}

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        all_train_labels = []
        all_train_predictions = []
        all_train_predictions_proba = []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_train_labels.extend(labels.cpu().numpy())
            all_train_predictions.extend(predicted.cpu().numpy())
            all_train_predictions_proba.extend(probabilities.cpu().detach().numpy())

        # Calculate training metrics
        all_train_labels = np.array(all_train_labels)
        all_train_predictions = np.array(all_train_predictions)
        all_train_predictions_proba = np.array(all_train_predictions_proba)
        train_metrics = calculate_metrics(all_train_labels, all_train_predictions, 
                                       all_train_predictions_proba, num_classes)
        
        # Validation phase
        val_metrics = evaluate(model, val_loader, device, num_classes)
        
        # Store metrics history
        history['train'].append({
            'loss': train_loss / len(train_loader),
            **train_metrics
        })
        history['val'].append(val_metrics)

        # Print epoch results
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("Training Metrics:")
        print(f"Loss: {train_loss/len(train_loader):.4f}")
        for metric, value in train_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
        
        print("\nValidation Metrics:")
        for metric, value in val_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
    
    return history

In [44]:
num_epochs = 5

optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [45]:
history = train(model, train_loader, dev_loader, criterion, optimizer, device, num_epochs)

Epoch 1/5: 100%|██████████| 49/49 [00:18<00:00,  2.59it/s]



Epoch 1/5
Training Metrics:
Loss: 1.3866
Accuracy: 0.2627
Precision: 0.2657
Recall: 0.2627
F1: 0.2525
Roc_auc: 0.5143

Validation Metrics:
Accuracy: 0.2433
Precision: 0.2962
Recall: 0.2433
F1: 0.2074
Roc_auc: 0.5377


Epoch 2/5: 100%|██████████| 49/49 [00:18<00:00,  2.59it/s]



Epoch 2/5
Training Metrics:
Loss: 1.3785
Accuracy: 0.2845
Precision: 0.2844
Recall: 0.2845
F1: 0.2801
Roc_auc: 0.5438

Validation Metrics:
Accuracy: 0.2701
Precision: 0.4569
Recall: 0.2701
F1: 0.1992
Roc_auc: 0.5697


Epoch 3/5: 100%|██████████| 49/49 [00:18<00:00,  2.59it/s]



Epoch 3/5
Training Metrics:
Loss: 1.3501
Accuracy: 0.3507
Precision: 0.3565
Recall: 0.3507
F1: 0.3441
Roc_auc: 0.6117

Validation Metrics:
Accuracy: 0.3381
Precision: 0.4926
Recall: 0.3381
F1: 0.3203
Roc_auc: 0.6207


Epoch 4/5: 100%|██████████| 49/49 [00:19<00:00,  2.57it/s]



Epoch 4/5
Training Metrics:
Loss: 1.2721
Accuracy: 0.4078
Precision: 0.4183
Recall: 0.4078
F1: 0.4072
Roc_auc: 0.6875

Validation Metrics:
Accuracy: 0.3959
Precision: 0.4711
Recall: 0.3959
F1: 0.4083
Roc_auc: 0.6740


Epoch 5/5: 100%|██████████| 49/49 [00:19<00:00,  2.57it/s]



Epoch 5/5
Training Metrics:
Loss: 1.1080
Accuracy: 0.5324
Precision: 0.5536
Recall: 0.5324
F1: 0.5341
Roc_auc: 0.7863

Validation Metrics:
Accuracy: 0.4247
Precision: 0.4470
Recall: 0.4247
F1: 0.4083
Roc_auc: 0.7099


In [46]:
import pandas as pd

pd.DataFrame(history['train'])

,loss,accuracy,precision,recall,f1,roc_auc
0,1.386598,0.262685,0.265676,0.262685,0.252458,0.514259
1,1.378525,0.284522,0.284378,0.284522,0.280115,0.543830
2,1.350147,0.350674,0.356452,0.350674,0.344080,0.611696
3,1.272121,0.407836,0.418336,0.407836,0.407249,0.687541
4,1.107979,0.532434,0.553638,0.532434,0.534137,0.786348


In [47]:
pd.DataFrame(history['val'])

,accuracy,precision,recall,f1,roc_auc
0,0.243299,0.296216,0.243299,0.207369,0.537720
1,0.270103,0.456855,0.270103,0.199217,0.569687
2,0.338144,0.492621,0.338144,0.320304,0.620732
3,0.395876,0.471083,0.395876,0.408338,0.673985
4,0.424742,0.447002,0.424742,0.408339,0.709863


In [48]:
evaluate(model, test_loader, device, num_classes)

{'accuracy': 0.45384615384615384,
 'precision': 0.47919288834055246,
 'recall': 0.45384615384615384,
 'f1': 0.43797895046513546,
 'roc_auc': 0.7197342214616704}

In [ ]:
9-